In [29]:
import numpy as np 
import open3d as o3d
import time
import logging
import math
import os

sample_directory = r"C:\Data\Temp\TC-Ghent\TrainingData\Labo-Beton\OUTPUT\P30"
# syntethic_directory = r"C:\Data\Temp\TC-Ghent\TrainingData\Labo-Beton\OUTPUT\SYNTH"
timestr = time.strftime("%Y%m%d-%H%M%S")
voxel_size_ref = 0.01 # voxel size for the downsampling
voxel_size_target = 0.01
init_radius = 0.15 # initial neirest neighbour search radius
c2c_treshold = 0.15 # All points with a c2c-distance larger than this treshold will be labeled as clutter
search_treshold = c2c_treshold # Max distance for normal comparison 

LOA_filename = "LOA_" + str(voxel_size_target) + "_" + timestr + ".txt"

LOA_location = sample_directory + "\\" + LOA_filename

result_directory = sample_directory + "\\Results_" + timestr

In [30]:
Zero_time = time.perf_counter()

print("Import meshes ...")

print("... CEILING ...")
mesh_ceiling = o3d.io.read_triangle_mesh(r"C:\Data\Temp\TC-Ghent\TrainingData\Labo-Beton\INPUT\BIM\OBJ\ceiling.obj")
print(mesh_ceiling)
ceiling_points = round(mesh_ceiling.get_surface_area() * 1000)
print("Convert Mesh to pointcloud")
meshpcd_ceiling = mesh_ceiling.sample_points_uniformly(number_of_points = ceiling_points, use_triangle_normal=True)
print(meshpcd_ceiling)
# o3d.visualization.draw_geometries([meshpcd_ceiling])
print("Voxel downsampling mesh with voxel size: %s m" % voxel_size_ref )
meshpcd_ceiling = meshpcd_ceiling.voxel_down_sample(voxel_size_ref)
print(meshpcd_ceiling)
# o3d.visualization.draw_geometries([meshpcd_ceiling])

print("... FLOOR ...")
mesh_floor = o3d.io.read_triangle_mesh(r"C:\Data\Temp\TC-Ghent\TrainingData\Labo-Beton\INPUT\BIM\OBJ\floor.obj")
print(mesh_floor)
print("Convert Mesh to pointcloud")
floor_points = round(mesh_floor.get_surface_area() * 1000)
meshpcd_floor = mesh_floor.sample_points_uniformly(number_of_points = floor_points, use_triangle_normal=True)
print(meshpcd_floor)
# o3d.visualization.draw_geometries([meshpcd_floor])
print("Voxel downsampling mesh with voxel size: %s m" % voxel_size_ref )
meshpcd_floor = meshpcd_floor.voxel_down_sample(voxel_size_ref)
print(meshpcd_floor)
# o3d.visualization.draw_geometries([meshpcd_floor])

print("... WALL ...")
mesh_wall = o3d.io.read_triangle_mesh(r"C:\Data\Temp\TC-Ghent\TrainingData\Labo-Beton\INPUT\BIM\OBJ\wall.obj")
print(mesh_wall)
print("Convert Mesh to pointcloud")
wall_points = round(mesh_wall.get_surface_area() * 1000)
meshpcd_wall = mesh_wall.sample_points_uniformly(number_of_points = wall_points, use_triangle_normal=True)
print(meshpcd_wall)
# o3d.visualization.draw_geometries([meshpcd_wall])
print("voxel downsampling mesh with voxel size: %s m" % voxel_size_ref )
meshpcd_wall = meshpcd_wall.voxel_down_sample(voxel_size_ref)
print(meshpcd_wall)
# o3d.visualization.draw_geometries([meshpcd_wall])

print("... BEAM ...")
mesh_beam = o3d.io.read_triangle_mesh(r"C:\Data\Temp\TC-Ghent\TrainingData\Labo-Beton\INPUT\BIM\OBJ\beam.obj")
print(mesh_beam)
print("Convert Mesh to pointcloud")
beam_points = round(mesh_beam.get_surface_area() * 1000)
meshpcd_beam = mesh_beam.sample_points_uniformly(number_of_points = beam_points, use_triangle_normal=True)
print(meshpcd_beam)
# o3d.visualization.draw_geometries([meshpcd_beam])
print("voxel downsampling mesh with voxel size: %s m" % voxel_size_ref )
meshpcd_beam = meshpcd_beam.voxel_down_sample(voxel_size_ref)
print(meshpcd_beam)
# o3d.visualization.draw_geometries([meshpcd_beam])

print("... COLUMN ...")
mesh_column = o3d.io.read_triangle_mesh(r"C:\Data\Temp\TC-Ghent\TrainingData\Labo-Beton\INPUT\BIM\OBJ\column.obj")
print(mesh_column)
print("Convert Mesh to pointcloud")
column_points = round(mesh_column.get_surface_area() * 1000)
meshpcd_column = mesh_column.sample_points_uniformly(number_of_points = column_points, use_triangle_normal=True)
print(meshpcd_column)
# o3d.visualization.draw_geometries([meshpcd_column])
print("voxel downsampling mesh with voxel size: %s m" % voxel_size_ref )
meshpcd_column = meshpcd_column.voxel_down_sample(voxel_size_ref)
print(meshpcd_column) 
o3d.visualization.draw_geometries([meshpcd_column])

t1 = time.perf_counter()
dt1 = t1 - Zero_time
print("Processed meshes in %s s" % dt1)
logging.info("Imported and sampled reference meshes in %s s.\n" % dt1)


Import meshes ...
... CEILING ...
TriangleMesh with 216 points and 72 triangles.
Convert Mesh to pointcloud
PointCloud with 856991 points.
Voxel downsampling mesh with voxel size: 0.01 m
PointCloud with 815546 points.
... FLOOR ...
TriangleMesh with 528 points and 176 triangles.
Convert Mesh to pointcloud
PointCloud with 1249367 points.
Voxel downsampling mesh with voxel size: 0.01 m
PointCloud with 1188949 points.
... WALL ...
TriangleMesh with 12762 points and 4254 triangles.
Convert Mesh to pointcloud
PointCloud with 2806117 points.
voxel downsampling mesh with voxel size: 0.01 m
PointCloud with 2664344 points.
... BEAM ...
TriangleMesh with 1872 points and 624 triangles.
Convert Mesh to pointcloud
PointCloud with 57241 points.
voxel downsampling mesh with voxel size: 0.01 m
PointCloud with 53904 points.
Processed meshes in 3.6026633000000174 s


In [31]:
t1 = time.perf_counter()

# Merge pointclouds from the meshes to 1 pointcloud and save labeles to a separate array
# FAST method
print("Merge refernce clouds to one pointcloud ...")

labels =[]
label_id = [ "Ceiling", "Floor", "Wall", "Beam", "Column", "Clutter"]

length_ceiling = len(meshpcd_ceiling.points)
i = 0
while i < length_ceiling:
    labels.append("0")
    i =  i+1

length_floor = len(meshpcd_floor.points)
i = 0
while i < length_floor:
    labels.append("1")
    i =  i+1

length_wall = len(meshpcd_wall.points)
i = 0
while i < length_wall:
    labels.append("2")
    i =  i+1

length_beam = len(meshpcd_beam.points)
i = 0
while i < length_beam:
    labels.append("3")
    i =  i+1

length_column = len(meshpcd_column.points)
i = 0
while i < length_column:
    labels.append("4")
    i =  i+1

# mesh_ceiling_t = o3d.t.geometry.PointCloud.from_legacy_pointcloud(meshpcd_ceiling)

meshpcd = o3d.geometry.PointCloud()
meshpcd.__iadd__(meshpcd_ceiling)
print(meshpcd)
meshpcd.__iadd__(meshpcd_floor)
print(meshpcd)
meshpcd.__iadd__(meshpcd_wall)
print(meshpcd)
meshpcd.__iadd__(meshpcd_beam)
print(meshpcd)
meshpcd.__iadd__(meshpcd_column)
print(meshpcd)
logging.info(str(meshpcd))

print("Reference cloud contains normals: %s" % meshpcd.has_normals())
print("%s / %s" %(len(labels), len(meshpcd.points)))
t2 = time.perf_counter()
dt2 = t2 - t1
print("Created reference cloud from meshes in %s s" % dt2)

Merge refernce clouds to one pointcloud ...
PointCloud with 815546 points.
PointCloud with 2004495 points.
PointCloud with 4668839 points.
PointCloud with 4722743 points.
Reference cloud contains normals: True
4722743 / 4722743
Created reference cloud from meshes in 0.8242900999999847 s


In [32]:
# print("Synthetic Cloud")
# pcd_filename = "SyntheticCloud_" + str(voxel_size_ref) + "_" + timestr + ".pcd"
# pcd_location = syntethic_directory +"\\" + pcd_filename
# o3d.io.write_point_cloud(pcd_location, meshpcd, print_progress=True)
# print("Saved to:  %s " % pcd_location)

In [33]:
# print(np.asarray(meshpcd.points[1:10]))

In [34]:
# o3d.visualization.draw_geometries([meshpcd])

In [35]:
t2 = time.perf_counter()

print("Load targeted pointcloudpoint cloud ...")

pointcloud_location = r"C:\Data\Temp\TC-Ghent\TrainingData\Labo-Beton\INPUT\POINTCLOUD\PCD\P30.pcd"
pcd = o3d.io.read_point_cloud(pointcloud_location)


t3 = time.perf_counter()
dt3 = t3 - t2
print("Target cloud loaded in %s s" % dt3)
print(pcd)

print("Downsampling pointcloud with voxel size: %s m" % voxel_size_target)
pcd = pcd.voxel_down_sample(voxel_size_target)
# tar_voxel.paint_uniform_color([0.5,0.5,0.5]) #Grey

t4 = time.perf_counter()
dt4 = t4 - t3


print("Downsampled the target pointcloud in %s s" % dt4)
print(pcd)
print("Target pointcloud has normals: %s " % pcd.has_normals())



Load targeted pointcloudpoint cloud ...
Target cloud loaded in 2.2115848999999344 s
PointCloud with 17827807 points.
Downsampling pointcloud with voxel size: 0.01 m
Downsampled the target pointcloud in 10.829095800000005 s
PointCloud with 10192273 points.
Target pointcloud has normals: False 


In [36]:
pcd.estimate_normals()

In [37]:
t4 = time.perf_counter()

c2c = pcd.compute_point_cloud_distance(meshpcd)

t5 = time.perf_counter()
dt5 = t5 - t4

print("Cloud 2 cloud calculated in %s s" % dt5)
print("%s / %s" %(len(c2c), len(pcd.points)))


Cloud 2 cloud calculated in 344.8356124999999 s
10192273 / 10192273


In [38]:
t5 = time.perf_counter()

# Filter pointcloud on c2c distances
# FAST method

i=0
percentage = 0
Inlier_1_indeces = []
Outlier_indeces = []

while i < len(c2c):
    if c2c[i] <= c2c_treshold:
        Inlier_1_indeces.append(i)
    elif c2c[i] > c2c_treshold and c2c[i] < 6:
        Outlier_indeces.append(i)

    progress = i/len(c2c)*100
    
    if progress >= percentage:
        timing = time.perf_counter()
        dtiming = timing - t5
        hours = math.floor(dtiming/3600)
        minutes = math.floor(dtiming/60 - hours*60)
        seconds = dtiming - hours*3600 - minutes*60
        print("progress: %s %% - %sh %smin %ss" %(round(progress),hours, minutes, seconds))
        percentage = percentage + 5
    i = i+1

# Inlier_pcd = pcd.select_by_index(Inlier_indeces)


t6 = time.perf_counter()
dt6 = t6 - t5
print("split in %s s" % dt6)

progress: 0 % - 0h 0min 0.035571699999991324s
progress: 5 % - 0h 0min 1.9982925000001615s
progress: 10 % - 0h 0min 4.083183599999984s
progress: 15 % - 0h 0min 6.247436600000128s
progress: 20 % - 0h 0min 8.277163100000053s
progress: 25 % - 0h 0min 10.346184100000073s
progress: 30 % - 0h 0min 12.184548599999971s
progress: 35 % - 0h 0min 14.2072462000001s
progress: 40 % - 0h 0min 16.086768800000073s
progress: 45 % - 0h 0min 18.219836800000166s
progress: 50 % - 0h 0min 20.37796010000011s
progress: 55 % - 0h 0min 22.736597800000027s
progress: 60 % - 0h 0min 25.122172800000044s
progress: 65 % - 0h 0min 27.411473500000056s
progress: 70 % - 0h 0min 29.50220730000001s
progress: 75 % - 0h 0min 31.592353700000103s
progress: 80 % - 0h 0min 33.56306110000014s
progress: 85 % - 0h 0min 35.13449020000007s
progress: 90 % - 0h 0min 37.26710160000016s
progress: 95 % - 0h 0min 38.98497590000011s
split in 40.32152340000016 s


De Dichtste mesh vinden 

Normaal van het punt vergelijken met de normaal van de mesh

Afwijkende normale ditchen 


In [39]:
t6 = time.perf_counter()

# Filter pointcloud on normals and assign labels.


i=0
percentage = 0
Inlier_2_indeces = []
Final_inlier_labels = []
Final_LOAs = []

print("creating a KD-tree")
meshpcd_tree = o3d.geometry.KDTreeFlann(meshpcd)#Create KD tree index

print("Finished creating a KD-tree")
progress = 0
count = 0
radius = init_radius

for i in Inlier_1_indeces:
    [k, idx, d] = meshpcd_tree.search_radius_vector_3d(pcd.points[i], init_radius) #Neighbour Search radius 10cm 
    # print("Selected points in the neighbourhood")
    # meshpcd_sample_points = np.asarray(meshpcd.points)[idx[:], :]
    # meshpcd_sample_normals = np.asarray(meshpcd.normals)[idx[:], :]
    Not_found = True
    i1=0
    while Not_found and i1 < len(idx) and len(idx) > 0:
        if np.abs(np.dot(np.asarray(pcd.normals[i]), np.asarray(meshpcd.normals)[idx[i1],:])) > 0.9 and np.abs(d[i1]) < c2c_treshold/5 or np.abs(np.dot(np.asarray(pcd.normals[i]), np.asarray(meshpcd.normals)[idx[i1],:])) > 0.7 and np.abs(d[i1]) < c2c_treshold/10:
            Not_found = False
            Inlier_2_indeces.append(i)
            Final_inlier_labels.append(labels[idx[i1]])
            Final_LOAs.append(d[i1]) 
        i1 = i1+1

    if Not_found:
        Outlier_indeces.append(i)

    count = count + 1
    progress = count/len(Inlier_1_indeces)*100
    
    if progress >= percentage:
        timing = time.perf_counter()
        dtiming = timing - t6
        hours = math.floor(dtiming/3600)
        minutes = math.floor(dtiming/60 - hours*60)
        seconds = dtiming - hours*3600 - minutes*60
        print("progress: %s %% - %sh %smin %ss" %(round(progress),hours, minutes, seconds))
        percentage = percentage + 5


print("extracting relevant points")

Final_inlier_pcd = pcd.select_by_index(Inlier_2_indeces)
print(Final_inlier_pcd)
Clutter_pcd = pcd.select_by_index(Outlier_indeces)
print(Clutter_pcd)

t7 = time.perf_counter()
dt7 = t7 - t6
print("filtered data in %s s" % dt7)
logging.info("Fine filtering took %s s" % dt7)

creating a KD-tree
Finished creating a KD-tree
progress: 0 % - 0h 0min 2.5836669000000256s
progress: 5 % - 0h 0min 20.51775100000009s
progress: 10 % - 0h 0min 38.0968476999999s
progress: 15 % - 0h 0min 55.88504869999997s
progress: 20 % - 0h 1min 13.414277200000015s
progress: 25 % - 0h 1min 31.619764399999895s
progress: 30 % - 0h 1min 47.456066599999986s
progress: 35 % - 0h 2min 2.5709285000000364s
progress: 40 % - 0h 2min 16.69724259999998s
progress: 45 % - 0h 2min 34.376372500000116s
progress: 50 % - 0h 2min 48.5362798000001s
progress: 55 % - 0h 3min 13.301010700000006s
progress: 60 % - 0h 3min 34.975927400000046s
progress: 65 % - 0h 3min 52.6412613s
progress: 70 % - 0h 4min 11.878172500000119s
progress: 75 % - 0h 4min 30.255810500000052s
progress: 80 % - 0h 4min 47.17792499999996s
progress: 85 % - 0h 5min 7.011828200000082s
progress: 90 % - 0h 5min 27.566237s
progress: 95 % - 0h 5min 46.33717610000008s
progress: 100 % - 0h 6min 6.484343600000102s
extracting relevant points
PointCloud

In [40]:
t7 = time.perf_counter()

i=0
percentage = 0

Class_0_indeces = []
Class_0_points = 0
Class_0_LOA30 = 0
Class_0_LOA20 = 0
Class_0_LOA10 = 0

Class_1_indeces = []
Class_1_points = 0
Class_1_LOA30 = 0
Class_1_LOA20 = 0
Class_1_LOA10 = 0

Class_2_indeces = []
Class_2_points = 0
Class_2_LOA30 = 0
Class_2_LOA20 = 0
Class_2_LOA10 = 0

Class_3_indeces = []
Class_3_points = 0
Class_3_LOA30 = 0
Class_3_LOA20 = 0
Class_3_LOA10 = 0

Class_4_indeces = []
Class_4_points = 0
Class_4_LOA30 = 0
Class_4_LOA20 = 0
Class_4_LOA10 = 0

Class_clutter_points = len(Clutter_pcd.points)

while i < len(np.asarray(Final_inlier_pcd.points)):
    if Final_inlier_labels[i] == "0":
        Class_0_indeces.append(i)
        Class_0_points = Class_0_points + 1
        dist = Final_LOAs[i]
        if dist <= 0.015:
            Class_0_LOA30 = Class_0_LOA30 + 1
        elif dist > 0.015 and dist <=0.05:
            Class_0_LOA20 = Class_0_LOA20 + 1
        elif dist > 0.05:
            Class_0_LOA10 = Class_0_LOA10 + 1

    elif Final_inlier_labels[i] == "1":
        Class_1_indeces.append(i)
        Class_1_points = Class_1_points + 1
        dist = Final_LOAs[i]
        if dist <= 0.015:
            Class_1_LOA30 = Class_1_LOA30 + 1
        elif dist > 0.015 and dist <=0.05:
            Class_1_LOA20 = Class_1_LOA20 + 1
        elif dist > 0.05:
            Class_1_LOA10 = Class_1_LOA10 + 1

    elif Final_inlier_labels[i] == "2":
        Class_2_indeces.append(i)
        Class_2_points = Class_2_points + 1
        dist = Final_LOAs[i]
        if dist <= 0.015:
            Class_2_LOA30 = Class_2_LOA30 + 1
        elif dist > 0.015 and dist <=0.05:
            Class_2_LOA20 = Class_2_LOA20 + 1
        elif dist > 0.05:
            Class_2_LOA10 = Class_2_LOA10 + 1

    elif Final_inlier_labels[i] == "3":
        Class_3_indeces.append(i)
        Class_3_points = Class_3_points + 1
        dist = Final_LOAs[i]
        if dist <= 0.015:
            Class_3_LOA30 = Class_3_LOA30 + 1
        elif dist > 0.015 and dist <=0.05:
            Class_3_LOA20 = Class_3_LOA20 + 1
        elif dist > 0.05:
            Class_3_LOA10 = Class_3_LOA10 + 1

    if Final_inlier_labels[i] == "4":
        Class_4_indeces.append(i)
        Class_4_points = Class_4_points + 1
        dist = Final_LOAs[i]
        if dist <= 0.015:
            Class_4_LOA30 = Class_4_LOA30 + 1
        elif dist > 0.015 and dist <=0.05:
            Class_4_LOA20 = Class_4_LOA20 + 1
        elif dist > 0.05:
            Class_4_LOA10 = Class_4_LOA10 + 1

    progress = i/len(np.asarray(Final_inlier_pcd.points))*100
    if progress >= percentage:
        timing = time.perf_counter()
        dtiming = timing - t7
        hours = math.floor(dtiming/3600)
        minutes = math.floor(dtiming/60 - hours*60)
        seconds = dtiming - hours*3600 - minutes*60
        print("progress: %s %% - %sh %smin %ss" %(round(progress),hours, minutes, seconds))
        percentage = percentage + 5
    i= i+1

print("...DONE...")
t8 = time.perf_counter()
dt8 = t8 - t7
print("calculated LOA specs in %s s" % dt8)


progress: 0 % - 0h 0min 0.040070099999866216s
progress: 5 % - 0h 0min 1.2425190999999813s
progress: 10 % - 0h 0min 2.436567299999979s
progress: 15 % - 0h 0min 3.6521874000000025s
progress: 20 % - 0h 0min 4.827905099999953s
progress: 25 % - 0h 0min 6.011885799999845s
progress: 30 % - 0h 0min 7.21694669999988s
progress: 35 % - 0h 0min 8.410321999999951s
progress: 40 % - 0h 0min 9.630640699999958s
progress: 45 % - 0h 0min 10.810164900000018s
progress: 50 % - 0h 0min 11.993280400000003s
progress: 55 % - 0h 0min 13.190439399999832s
progress: 60 % - 0h 0min 14.413202100000035s
progress: 65 % - 0h 0min 15.61598069999991s
progress: 70 % - 0h 0min 16.81661710000003s
progress: 75 % - 0h 0min 18.00145879999991s
progress: 80 % - 0h 0min 19.179852499999924s
progress: 85 % - 0h 0min 20.378032099999928s
progress: 90 % - 0h 0min 21.595569699999942s
progress: 95 % - 0h 0min 22.803336700000045s
...DONE...
calculated LOA specs in 23.99203909999983 s


In [41]:
t8 = time.perf_counter()

result_directory = sample_directory + "\\Results_" + timestr
if not os.path.exists(result_directory):
    os.mkdir(result_directory)


print("Class 0:  %s " % label_id[0])
Class_0_pcd = Final_inlier_pcd.select_by_index(Class_0_indeces)
print(Class_0_pcd)
Class_0_filename = "Cloud_P30_" + str(voxel_size_target) + "_" + label_id[0] + ".pcd"
Class_0_location = result_directory +"\\" + Class_0_filename
o3d.io.write_point_cloud(Class_0_location, Class_0_pcd, print_progress=True)
print("Saved to:  %s " % Class_0_location)

print("Class 1:  %s " % label_id[1])
Class_1_pcd = Final_inlier_pcd.select_by_index(Class_1_indeces)
print(Class_1_pcd)
Class_1_filename = "Cloud_P30_" + str(voxel_size_target) + "_" + label_id[1] + ".pcd"
Class_1_location = result_directory +"\\" + Class_1_filename
o3d.io.write_point_cloud(Class_1_location, Class_1_pcd, print_progress=True)
print("Saved to:  %s " % Class_1_location)

print("Class 2:  %s " % label_id[2])
Class_2_pcd = Final_inlier_pcd.select_by_index(Class_2_indeces)
print(Class_2_pcd)
Class_2_filename = "Cloud_P30_" + str(voxel_size_target) + "_" + label_id[2] + ".pcd"
Class_2_location = result_directory +"\\" + Class_2_filename
o3d.io.write_point_cloud(Class_2_location, Class_2_pcd, print_progress=True)
print("Saved to:  %s " % Class_2_location)

print("Class 3:  %s " % label_id[3])
Class_3_pcd = Final_inlier_pcd.select_by_index(Class_3_indeces)
print(Class_3_pcd)
Class_3_filename = "Cloud_P30_" + str(voxel_size_target) + "_" + label_id[3] + ".pcd"
Class_3_location = result_directory +"\\" + Class_3_filename
o3d.io.write_point_cloud(Class_3_location, Class_3_pcd, print_progress=True)
print("Saved to:  %s " % Class_3_location)

print("Class 4:  %s " % label_id[4])
Class_4_pcd = Final_inlier_pcd.select_by_index(Class_4_indeces)
print(Class_4_pcd)
Class_4_filename = "Cloud_P30_" + str(voxel_size_target) + "_" + label_id[4] + ".pcd"
Class_4_location = result_directory +"\\" + Class_4_filename
o3d.io.write_point_cloud(Class_4_location, Class_4_pcd, print_progress=True)
print("Saved to:  %s " % Class_4_location)

print("Class Clutter")
print(Clutter_pcd)
Class_Clutter_filename = "Cloud_P30_" + str(voxel_size_target) + "_Clutter"  + ".pcd"
Class_Clutter_location = result_directory +"\\" + Class_Clutter_filename
o3d.io.write_point_cloud(Class_Clutter_location, Clutter_pcd, print_progress=True)
print("Saved to:  %s " % Class_Clutter_location)

print("General Cloud")
pcd_filename = "GeneralCloud_P30_" + str(voxel_size_target) + "_" + timestr + ".pcd"
pcd_location = result_directory +"\\" + pcd_filename
o3d.io.write_point_cloud(pcd_location, Final_inlier_pcd, print_progress=True)
print("Saved to:  %s " % pcd_location)

t9 = time.perf_counter()
dt9 = t9 - t8
print("Saved clouds in %s s" % dt9)

Class 0:  Ceiling 
PointCloud with 751729 points.
Saved to:  C:\Data\Temp\TC-Ghent\TrainingData\Labo-Dubo\OUTPUT\M6\Results_20210902-150733\Cloud_M6_0.01_Ceiling.pcd 
Class 1:  Floor 
PointCloud with 780946 points.
Saved to:  C:\Data\Temp\TC-Ghent\TrainingData\Labo-Dubo\OUTPUT\M6\Results_20210902-150733\Cloud_M6_0.01_Floor.pcd 
Class 2:  Wall 
PointCloud with 1794022 points.
Saved to:  C:\Data\Temp\TC-Ghent\TrainingData\Labo-Dubo\OUTPUT\M6\Results_20210902-150733\Cloud_M6_0.01_Wall.pcd 
Class 3:  Beam 
PointCloud with 131465 points.
Saved to:  C:\Data\Temp\TC-Ghent\TrainingData\Labo-Dubo\OUTPUT\M6\Results_20210902-150733\Cloud_M6_0.01_Beam.pcd 
Class Clutter
PointCloud with 4084073 points.
Saved to:  C:\Data\Temp\TC-Ghent\TrainingData\Labo-Dubo\OUTPUT\M6\Results_20210902-150733\Cloud_M6_0.01_Clutter.pcd 
General Cloud
Saved to:  C:\Data\Temp\TC-Ghent\TrainingData\Labo-Dubo\OUTPUT\M6\Results_20210902-150733\GeneralCloud_M6_0.01_20210902-150733.pcd 
Saved clouds in 2.3441358000000037 s


In [42]:
# o3d.visualization.draw_geometries([Final_inlier_pcd,Clutter_pcd])